In [1]:
%%pyspark
df = spark.read.load('abfss://files@datalake8fve9ro.dfs.core.windows.net/products/products.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

StatementMeta(spark8fve9ro, 0, 2, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fe7066f0-4a44-4de9-8b52-8142eb92fdd4)

In [2]:
 delta_table_path = "/delta/products-delta"
 df.write.format("delta").save(delta_table_path)
print("df written")

StatementMeta(spark8fve9ro, 0, 3, Finished, Available, Finished)

df written


In [3]:
 from delta.tables import *
 from pyspark.sql.functions import *

 # Create a deltaTable object
 deltaTable = DeltaTable.forPath(spark, delta_table_path)

 # Update the table (reduce price of product 771 by 10%)
 deltaTable.update(
     condition = "ProductID == 771",
     set = { "ListPrice": "ListPrice * 0.9" })

 # View the updated data as a dataframe
 deltaTable.toDF().show(10)


StatementMeta(spark8fve9ro, 0, 4, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [4]:
 new_df = spark.read.format("delta").load(delta_table_path)
 new_df.show(10)


StatementMeta(spark8fve9ro, 0, 5, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes| 3059.991|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [5]:
 new_df = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
 new_df.show(10)


StatementMeta(spark8fve9ro, 0, 6, Finished, Available, Finished)

+---------+--------------------+--------------+---------+
|ProductID|         ProductName|      Category|ListPrice|
+---------+--------------------+--------------+---------+
|      771|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      772|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      773|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      774|Mountain-100 Silv...|Mountain Bikes|3399.9900|
|      775|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      776|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      777|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      778|Mountain-100 Blac...|Mountain Bikes|3374.9900|
|      779|Mountain-200 Silv...|Mountain Bikes|2319.9900|
|      780|Mountain-200 Silv...|Mountain Bikes|2319.9900|
+---------+--------------------+--------------+---------+
only showing top 10 rows



In [6]:
 deltaTable.history(10).show(20, False, True)


StatementMeta(spark8fve9ro, 0, 7, Finished, Available, Finished)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 version             | 1                                                                                                                                                                                                                                     
 timestamp           | 2025-05-15 16:58:57.36                                                                                                                                                                                                                
 userId              | null                                                                                                                                                                                                                   

In [7]:
 spark.sql("CREATE DATABASE AdventureWorks")
 spark.sql("CREATE TABLE AdventureWorks.ProductsExternal USING DELTA LOCATION '{0}'".format(delta_table_path))
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsExternal").show(truncate=False)


StatementMeta(spark8fve9ro, 0, 8, Finished, Available, Finished)

+----------------------------+-----------------------------------------------------------------------+-------+
|col_name                    |data_type                                                              |comment|
+----------------------------+-----------------------------------------------------------------------+-------+
|ProductID                   |string                                                                 |null   |
|ProductName                 |string                                                                 |null   |
|Category                    |string                                                                 |null   |
|ListPrice                   |string                                                                 |null   |
|                            |                                                                       |       |
|# Detailed Table Information|                                                                       |       |
|

In [8]:
 %%sql
 USE AdventureWorks;
 SELECT * FROM ProductsExternal;

StatementMeta(spark8fve9ro, 0, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [9]:
 df.write.format("delta").saveAsTable("AdventureWorks.ProductsManaged")
 spark.sql("DESCRIBE EXTENDED AdventureWorks.ProductsManaged").show(truncate=False)


StatementMeta(spark8fve9ro, 0, 11, Finished, Available, Finished)

+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                       |comment|
+----------------------------+--------------------------------------------------------------------------------------------------------------------------------+-------+
|ProductID                   |string                                                                                                                          |null   |
|ProductName                 |string                                                                                                                          |null   |
|Category                    |string                                                                                                                          |n

In [10]:
 %%sql
 USE AdventureWorks;
 SELECT * FROM ProductsManaged;

StatementMeta(spark8fve9ro, 0, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [11]:
 %%sql
 USE AdventureWorks;
 SHOW TABLES;


StatementMeta(spark8fve9ro, 0, 15, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 2 rows and 3 fields>

In [12]:
 %%sql
 USE AdventureWorks;
 DROP TABLE IF EXISTS ProductsExternal;
 DROP TABLE IF EXISTS ProductsManaged;


StatementMeta(spark8fve9ro, 0, 18, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [13]:
 %%sql
 USE AdventureWorks;
 CREATE TABLE Products
 USING DELTA
 LOCATION '/delta/products-delta';


StatementMeta(spark8fve9ro, 0, 20, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [14]:
 %%sql
 USE AdventureWorks;
 SELECT * FROM Products;

StatementMeta(spark8fve9ro, 0, 22, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 295 rows and 4 fields>

In [15]:
 from notebookutils import mssparkutils
 from pyspark.sql.types import *
 from pyspark.sql.functions import *

 # Create a folder
 inputPath = '/data/'
 mssparkutils.fs.mkdirs(inputPath)

 # Create a stream that reads data from the folder, using a JSON schema
 jsonSchema = StructType([
 StructField("device", StringType(), False),
 StructField("status", StringType(), False)
 ])
 iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

 # Write some event data to the folder
 device_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"ok"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''
 mssparkutils.fs.put(inputPath + "data.txt", device_data, True)
 print("Source stream created...")


StatementMeta(spark8fve9ro, 0, 23, Finished, Available, Finished)

Source stream created...


In [16]:
 # Write the stream to a delta table
 delta_stream_table_path = '/delta/iotdevicedata'
 checkpointpath = '/delta/checkpoint'
 deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
 print("Streaming to delta sink...")


StatementMeta(spark8fve9ro, 0, 24, Finished, Available, Finished)

Streaming to delta sink...


In [17]:
 # Read the data in delta format into a dataframe
 df = spark.read.format("delta").load(delta_stream_table_path)
 display(df)


StatementMeta(spark8fve9ro, 0, 25, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 122caf3f-8594-44e8-b4f6-0d5688612374)

In [18]:
 # create a catalog table based on the streaming sink
 spark.sql("CREATE TABLE IotDeviceData USING DELTA LOCATION '{0}'".format(delta_stream_table_path))

StatementMeta(spark8fve9ro, 0, 26, Finished, Available, Finished)

DataFrame[]

In [19]:
 %%sql
 SELECT * FROM IotDeviceData;

StatementMeta(spark8fve9ro, 0, 27, Finished, Available, Finished)

<Spark SQL result set with 9 rows and 2 fields>

In [20]:
 # Add more data to the source stream
 more_data = '''{"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"ok"}
 {"device":"Dev1","status":"error"}
 {"device":"Dev2","status":"error"}
 {"device":"Dev1","status":"ok"}'''

 mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)


StatementMeta(spark8fve9ro, 0, 28, Finished, Available, Finished)

True

In [21]:
 %%sql
 SELECT * FROM IotDeviceData;

StatementMeta(spark8fve9ro, 0, 29, Finished, Available, Finished)

<Spark SQL result set with 16 rows and 2 fields>

In [23]:
 deltastream.stop()
print("Stream stopped")

StatementMeta(spark8fve9ro, 0, 31, Finished, Available, Finished)

Stream stopped


In [ ]:
%%sql
-- files/delta/products-delta > Select TOP 100 rows > Delta format
-- This is auto-generated code
SELECT
    TOP 100 *
FROM
    OPENROWSET(
        BULK 'https://datalake8fve9ro.dfs.core.windows.net/files/delta/products-delta/',
        FORMAT = 'DELTA'
    ) AS [result]


In [ ]:
%%sql
-- Run the code and observe that you can also use the serverless SQL pool to query Delta Lake data in catalog tables that are defined the Spark metastore.
 USE AdventureWorks;
 SELECT * FROM Products;